In [79]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import keras.preprocessing
import tensorflow as tf
import numpy as np
import pandas as pd
import re

In [17]:
data_url = 'https://raw.githubusercontent.com/DavidCarricondo/Custom-database-goodreads-reviews/master/DATA/goodreads_reviews_clean.csv'
data = pd.read_csv(data_url)
data.head()

,review,grade,lang,score,sentiment
0,"The statue of Hadrian, the 14th Emperor of the...",it was amazing,en,4,2
1,It's hard to know what to think about this boo...,really liked it,en,3,2
2,"If you're into stuff like this, you can read t...",it was amazing,en,4,2
3,Has Kenzaburo Oe ever visited Japan? Ban this ...,it was amazing,en,4,2
4,Sadza (typical cornmeal staple food in Zimbabw...,it was amazing,en,4,2


Shuffle the data and split the train and test set (the test set will be subsequently split into validation and test sets)

In [177]:
data = data.sample(frac=1).reset_index(drop=True)

In [235]:
train_size = len(data) * 90//100
data_train = data[:train_size]
data_test = data[train_size:]

In [114]:
import tensorflow_datasets as tfds
datasets, info = tfds.load('imdb_reviews', as_supervised=True, with_info=True)
train_size = info.splits['train'].num_examples

Shuffling and writing examples to /home/david/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteAKEDCB/imdb_reviews-train.tfrecord


Shuffling and writing examples to /home/david/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteAKEDCB/imdb_reviews-test.tfrecord


Shuffling and writing examples to /home/david/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteAKEDCB/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /home/david/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [212]:
type(datasets['train'])
datasets['train']

<PrefetchDataset shapes: ((), ()), types: (tf.string, tf.int64)>

**Following homl from pg 536:**

Transform data to tensors, then to tensorflow Dataset, and zip it into a prefetch Dataset:

In [247]:
rev_tensor = tf.data.Dataset.from_tensor_slices([tf.convert_to_tensor(e) for e in data_train.review])
sent_tensor = tf.data.Dataset.from_tensor_slices([tf.convert_to_tensor(e, dtype=tf.int64) for e in data_train.sentiment])

In [248]:
data_tensor = tf.data.Dataset.zip((rev_tensor, sent_tensor)).prefetch(1)

In [239]:
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 500)
    X_batch = tf.strings.regex_replace(X_batch, b"<br\\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

In [241]:
from collections import Counter

vocabulary = Counter()
for X_batch, y_batch in data_tensor.batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

In [242]:
vocabulary.most_common()[:3]

[(b'<pad>', 493320), (b'the', 69667), (b'of', 44835)]

<table>
<td> 
<img src="images/attn_model.png" style="width:500;height:500px;"> <br>
</td> 
<td> 
<img src="images/attn_mechanism.png" style="width:500;height:500px;"> <br>
</td> 
</table>
<caption><center> **Figure 1**: Neural machine translation with attention</center></caption>